# Memgraph QA chain | Memgraph QA チェーン

このノートブックは、LLMsを使用して[Memgraph](https://github.com/memgraph/memgraph)データベースに自然言語インターフェースを提供する方法を示しています。このチュートリアルを完了するためには、[Docker](https://www.docker.com/get-started/)と[Python 3.x](https://www.python.org/)のインストールが必要です。

> This notebook shows how to use LLMs to provide a natural language interface to a [Memgraph](https://github.com/memgraph/memgraph) database. To complete this tutorial, you will need [Docker](https://www.docker.com/get-started/) and [Python 3.x](https://www.python.org/) installed.

このチュートリアルを進めるためには、動作中のMemgraphインスタンスがあることを確認してください。以下のスクリプトを実行すれば、ローカルのDockerコンテナでMemgraphをダウンロードして実行できます：

> To follow along with this tutorial, ensure you have a running Memgraph instance. You can download and run it in a local Docker container by executing the following script:

```
docker run \
    -it \
    -p 7687:7687 \
    -p 7444:7444 \
    -p 3000:3000 \
    -e MEMGRAPH="--bolt-server-name-for-init=Neo4j/" \
    -v mg_lib:/var/lib/memgraph memgraph/memgraph-platform
```

データベースが起動するまで数秒待つ必要があります。プロセスが正常に完了すれば、以下のようなものが表示されるはずです：

> You will need to wait a few seconds for the database to start. If the process completes successfully, you should see something like this:

```
mgconsole X.X
Connected to 'memgraph://127.0.0.1:7687'
Type :help for shell usage
Quit the shell by typing Ctrl-D(eof) or :quit
memgraph>
```

これでMemgraphで遊び始めることができます！

> Now you can start playing with Memgraph!



まず、必要なパッケージをインストールしてインポートしましょう。適切な権限を確保するために、パッケージマネージャーである[pip](https://pip.pypa.io/en/stable/installation/)を`--user`フラグと共に使用します。Python 3.4またはそれ以降のバージョンをインストールしている場合、pipはデフォルトで含まれています。以下のコマンドを使用して、必要なパッケージをすべてインストールできます：

> Begin by installing and importing all the necessary packages. We'll use the package manager called [pip](https://pip.pypa.io/en/stable/installation/), along with the `--user` flag, to ensure proper permissions. If you've installed Python 3.4 or a later version, pip is included by default. You can install all the required packages using the following command:



In [ ]:
pip install langchain openai neo4j gqlalchemy --user

このノートブックに用意されているコードブロックを実行するか、またはMemgraphとLangChainを実験するために別のPythonファイルを使用することができます。

> You can either run the provided code blocks in this notebook or use a separate Python file to experiment with Memgraph and LangChain.



In [ ]:
import os

from gqlalchemy import Memgraph
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI
from langchain.graphs import MemgraphGraph
from langchain.prompts import PromptTemplate

私たちは、MemgraphデータベースとPythonスクリプトとの接続を確立するために、Pythonライブラリの[GQLAlchemy](https://github.com/memgraph/gqlalchemy)を利用しています。クエリを実行するために、以下のようにMemgraphインスタンスをセットアップすることができます：

> We're utilizing the Python library [GQLAlchemy](https://github.com/memgraph/gqlalchemy) to establish a connection between our Memgraph database and Python script. To execute queries, we can set up a Memgraph instance as follows:



In [ ]:
memgraph = Memgraph(host="127.0.0.1", port=7687)

## Populating the database | データベースの充填

Cypherクエリ言語を使用して、新しく空のデータベースに簡単にデータを追加することができます。すべての行をすぐに理解できなくても心配はいりません。Cypherについては[こちら](https://memgraph.com/docs/cypher-manual/)のドキュメントから学ぶことができます。以下のスクリプトを実行すると、データベースにシーディングクエリが実行され、ビデオゲームに関するデータが追加されます。これには、パブリッシャー、利用可能なプラットフォーム、ジャンルなどの詳細が含まれます。このデータは、私たちの作業の出発点として役立ちます。

> You can effortlessly populate your new, empty database using the Cypher query language. Don't worry if you don't grasp every line just yet, you can learn Cypher from the documentation [here](https://memgraph.com/docs/cypher-manual/). Running the following script will execute a seeding query on the database, giving us data about a video game, including details like the publisher, available platforms, and genres. This data will serve as a basis for our work.



In [ ]:
# Creating and executing the seeding query
query = """
    MERGE (g:Game {name: "Baldur's Gate 3"})
    WITH g, ["PlayStation 5", "Mac OS", "Windows", "Xbox Series X/S"] AS platforms,
            ["Adventure", "Role-Playing Game", "Strategy"] AS genres
    FOREACH (platform IN platforms |
        MERGE (p:Platform {name: platform})
        MERGE (g)-[:AVAILABLE_ON]->(p)
    )
    FOREACH (genre IN genres |
        MERGE (gn:Genre {name: genre})
        MERGE (g)-[:HAS_GENRE]->(gn)
    )
    MERGE (p:Publisher {name: "Larian Studios"})
    MERGE (g)-[:PUBLISHED_BY]->(p);
"""

memgraph.execute(query)

## Refresh graph schema | グラフスキーマを更新する



以下のスクリプトを使用して、Memgraph-LangChainグラフをインスタンス化する準備が整いました。このインターフェースにより、LangChainを使用してデータベースにクエリを実行し、LLMを介してCypherクエリを生成するために必要なグラフスキーマを自動的に作成できます。

> You're all set to instantiate the Memgraph-LangChain graph using the following script. This interface will allow us to query our database using LangChain, automatically creating the required graph schema for generating Cypher queries through LLM.



In [ ]:
graph = MemgraphGraph(url="bolt://localhost:7687", username="", password="")

必要に応じて、以下の手順でグラフスキーマを手動でリフレッシュすることができます。

> If necessary, you can manually refresh the graph schema as follows.



In [ ]:
graph.refresh_schema()

データに慣れ親しむとともに、更新されたグラフスキーマを確認するために、次のステートメントを使用して印刷できます。

> To familiarize yourself with the data and verify the updated graph schema, you can print it using the following statement.



In [ ]:
print(graph.schema)

```
Node properties are the following:
Node name: 'Game', Node properties: [{'property': 'name', 'type': 'str'}]
Node name: 'Platform', Node properties: [{'property': 'name', 'type': 'str'}]
Node name: 'Genre', Node properties: [{'property': 'name', 'type': 'str'}]
Node name: 'Publisher', Node properties: [{'property': 'name', 'type': 'str'}]

Relationship properties are the following:

The relationships are the following:
['(:Game)-[:AVAILABLE_ON]->(:Platform)']
['(:Game)-[:HAS_GENRE]->(:Genre)']
['(:Game)-[:PUBLISHED_BY]->(:Publisher)']
```



## Querying the database | データベースの問い合わせ



OpenAI APIとやり取りするためには、Pythonの[os](https://docs.python.org/3/library/os.html)パッケージを使用して、APIキーを環境変数として設定する必要があります。これにより、リクエストの適切な認証が保証されます。APIキーの取得方法についての詳細は[こちら](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key)で確認できます。

> To interact with the OpenAI API, you must configure your API key as an environment variable using the Python [os](https://docs.python.org/3/library/os.html) package. This ensures proper authorization for your requests. You can find more information on obtaining your API key [here](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key).



In [ ]:
os.environ["OPENAI_API_KEY"] = "your-key-here"

以下のスクリプトを使用してグラフチェーンを作成し、あなたのグラフデータに基づいた質問応答プロセスで利用します。デフォルトではGPT-3.5-turboが使用されますが、Cypherクエリと結果の顕著な改善のために、[GPT-4](https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4)など他のモデルの実験も検討する価値があります。以前に設定したキーを利用してOpenAIチャットを使用します。予測可能で一貫性のある回答を得るために、温度をゼロに設定します。また、Memgraph-LangChainグラフを使用し、デフォルトでFalseになっているverboseパラメータをTrueに設定することで、クエリ生成に関するより詳細なメッセージを受け取ることができます。

> You should create the graph chain using the following script, which will be utilized in the question-answering process based on your graph data. While it defaults to GPT-3.5-turbo, you might also consider experimenting with other models like [GPT-4](https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4) for notably improved Cypher queries and outcomes. We'll utilize the OpenAI chat, utilizing the key you previously configured. We'll set the temperature to zero, ensuring predictable and consistent answers. Additionally, we'll use our Memgraph-LangChain graph and set the verbose parameter, which defaults to False, to True to receive more detailed messages regarding query generation.



In [ ]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, model_name="gpt-3.5-turbo"
)

さあ、質問を始めてください！

> Now you can start asking questions!



In [ ]:
response = chain.run("Which platforms is Baldur's Gate 3 available on?")
print(response)

```
> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (g:Game {name: 'Baldur\'s Gate 3'})-[:AVAILABLE_ON]->(p:Platform)
RETURN p.name
Full Context:
[{'p.name': 'PlayStation 5'}, {'p.name': 'Mac OS'}, {'p.name': 'Windows'}, {'p.name': 'Xbox Series X/S'}]

> Finished chain.
Baldur's Gate 3 is available on PlayStation 5, Mac OS, Windows, and Xbox Series X/S.
```



In [ ]:
response = chain.run("Is Baldur's Gate 3 available on Windows?")
print(response)

```
> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Game {name: 'Baldur\'s Gate 3'})-[:AVAILABLE_ON]->(:Platform {name: 'Windows'})
RETURN true
Full Context:
[{'true': True}]

> Finished chain.
Yes, Baldur's Gate 3 is available on Windows.
```



## Chain modifiers | チェーン修飾子



チェーンの挙動を変更し、より多くのコンテキストや追加情報を得るためには、チェーンのパラメータを変更することができます。

> To modify the behavior of your chain and obtain more context or additional information, you can modify the chain's parameters.



#### Return direct query results | 直接的なクエリ結果を返す

`return_direct` 修飾子は、実行された Cypher クエリの直接的な結果を返すか、処理された自然言語の応答を返すかを指定します。

> The `return_direct` modifier specifies whether to return the direct results of the executed Cypher query or the processed natural language response.



In [ ]:
# Return the result of querying the graph directly
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, return_direct=True
)

In [ ]:
response = chain.run("Which studio published Baldur's Gate 3?")
print(response)

```
> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Game {name: 'Baldur\'s Gate 3'})-[:PUBLISHED_BY]->(p:Publisher)
RETURN p.name

> Finished chain.
[{'p.name': 'Larian Studios'}]
```



#### Return query intermediate steps | クエリの中間ステップを返す

`return_intermediate_steps` チェーン修飾子は、初期のクエリ結果に加えて、クエリの中間ステップも含めることで、返されるレスポンスを強化します。

> The `return_intermediate_steps` chain modifier enhances the returned response by including the intermediate steps of the query in addition to the initial query result.



In [ ]:
# Return all the intermediate steps of query execution
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, return_intermediate_steps=True
)

In [ ]:
response = chain("Is Baldur's Gate 3 an Adventure game?")
print(f"Intermediate steps: {response['intermediate_steps']}")
print(f"Final response: {response['result']}")

```
> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (g:Game {name: 'Baldur\'s Gate 3'})-[:HAS_GENRE]->(genre:Genre {name: 'Adventure'})
RETURN g, genre
Full Context:
[{'g': {'name': "Baldur's Gate 3"}, 'genre': {'name': 'Adventure'}}]

> Finished chain.
Intermediate steps: [{'query': "MATCH (g:Game {name: 'Baldur\\'s Gate 3'})-[:HAS_GENRE]->(genre:Genre {name: 'Adventure'})\nRETURN g, genre"}, {'context': [{'g': {'name': "Baldur's Gate 3"}, 'genre': {'name': 'Adventure'}}]}]
Final response: Yes, Baldur's Gate 3 is an Adventure game.
```



#### Limit the number of query results | クエリ結果の数を制限する

`top_k` 修飾子は、クエリ結果の最大数を制限したい場合に使用できます。

> The `top_k` modifier can be used when you want to restrict the maximum number of query results.



In [ ]:
# Limit the maximum number of results returned by query
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, top_k=2
)

In [ ]:
response = chain.run("What genres are associated with Baldur's Gate 3?")
print(response)

```
> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Game {name: 'Baldur\'s Gate 3'})-[:HAS_GENRE]->(g:Genre)
RETURN g.name
Full Context:
[{'g.name': 'Adventure'}, {'g.name': 'Role-Playing Game'}]

> Finished chain.
Baldur's Gate 3 is associated with the genres Adventure and Role-Playing Game.
```



# Advanced querying | 高度なクエリ



ソリューションの複雑さが増すにつれて、慎重な取り扱いを必要とする様々なユースケースに遭遇する可能性があります。アプリケーションのスケーラビリティを確保することは、何の問題もなくスムーズなユーザーフローを維持するために不可欠です。

> As the complexity of your solution grows, you might encounter different use-cases that require careful handling. Ensuring your application's scalability is essential to maintain a smooth user flow without any hitches.



もう一度チェーンをインスタンス化し、ユーザーが潜在的に尋ねるかもしれない質問に挑戦してみましょう。

> Let's instantiate our chain once again and attempt to ask some questions that users might potentially ask.



In [ ]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True, model_name="gpt-3.5-turbo"
)

In [ ]:
response = chain.run("Is Baldur's Gate 3 available on PS5?")
print(response)

```
> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (g:Game {name: 'Baldur\'s Gate 3'})-[:AVAILABLE_ON]->(p:Platform {name: 'PS5'})
RETURN g.name, p.name
Full Context:
[]

> Finished chain.
I'm sorry, but I don't have the information to answer your question.
```



生成されたCypherクエリは正しそうですが、応答として情報を受け取っていません。これは、LLMを使用する際の一般的な課題を示しています - ユーザーがクエリをどのように表現するかと、データがどのように格納されているかの間の不一致です。この場合、ユーザーの認識と実際のデータ格納の違いが不一致を引き起こす可能性があります。プロンプトの洗練、つまりモデルのプロンプトを磨き込んでこれらの違いをより良く把握するプロセスは、この問題に対処する効率的な解決策です。プロンプトの洗練を通じて、モデルは正確で関連性の高いクエリを生成する能力を高め、求められるデータの成功した取得につながります。

> The generated Cypher query looks fine, but we didn't receive any information in response. This illustrates a common challenge when working with LLMs - the misalignment between how users phrase queries and how data is stored. In this case, the difference between user perception and the actual data storage can cause mismatches. Prompt refinement, the process of honing the model's prompts to better grasp these distinctions, is an efficient solution that tackles this issue. Through prompt refinement, the model gains increased proficiency in generating precise and pertinent queries, leading to the successful retrieval of the desired data.



### Prompt refinement | プロンプトの改善



この問題に対処するために、QAチェーンの初期Cypherプロンプトを調整できます。これには、私たちのケースであるPS5のような特定のプラットフォームをユーザーがどのように参照するかについてLLMに指示を加えることが含まれます。これはLangChainの[PromptTemplate](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/)を使用して、修正された初期プロンプトを作成することで達成します。その後、この修正されたプロンプトは、改良されたMemgraph-LangChainインスタンスに引数として供給されます。

> To address this, we can adjust the initial Cypher prompt of the QA chain. This involves adding guidance to the LLM on how users can refer to specific platforms, such as PS5 in our case. We achieve this using the LangChain [PromptTemplate](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/), creating a modified initial prompt. This modified prompt is then supplied as an argument to our refined Memgraph-LangChain instance.



In [ ]:
CYPHER_GENERATION_TEMPLATE = """
Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
If the user asks about PS5, Play Station 5 or PS 5, that is the platform called PlayStation 5.

The question is:
{question}
"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

In [ ]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    graph=graph,
    verbose=True,
    model_name="gpt-3.5-turbo",
)

In [ ]:
response = chain.run("Is Baldur's Gate 3 available on PS5?")
print(response)

```
> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (g:Game {name: 'Baldur\'s Gate 3'})-[:AVAILABLE_ON]->(p:Platform {name: 'PlayStation 5'})
RETURN g.name, p.name
Full Context:
[{'g.name': "Baldur's Gate 3", 'p.name': 'PlayStation 5'}]

> Finished chain.
Yes, Baldur's Gate 3 is available on PlayStation 5.
```



プラットフォームの命名に関するガイダンスを含んだ改訂された初期のCypherプロンプトを使用することで、ユーザーのクエリにより密接に一致する正確で関連性の高い結果を得られるようになりました。

> Now, with the revised initial Cypher prompt that includes guidance on platform naming, we are obtaining accurate and relevant results that align more closely with user queries.



この手法により、QAチェーンをさらに改良することができます。追加のプロンプト精緻化データを手間なくチェーンに組み込むことで、アプリの全体的なユーザー体験を高めることが可能です。

> This approach allows for further improvement of your QA chain. You can effortlessly integrate extra prompt refinement data into your chain, thereby enhancing the overall user experience of your app.

